In [ ]:
import pandas as pd
import requests
import sqlalchemy
import my_stuff
from datetime import timedelta, datetime

In [ ]:
def get_connection_string():
    schema = "gans"
    host = "127.0.0.1"
    user = "root"
    password = my_stuff.my_sql_password
    port = 3306

    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    return connection_string

In [ ]:
def get_airports():
    connection_string = get_connection_string()
    airports_df = pd.read_sql('airport',con=connection_string)
    return airports_df

In [ ]:
def get_and_store_flights(airports_df):


    connection_string = get_connection_string()

    now = datetime.now()
    tomorrow = now + timedelta(days=1)
    tomorrow = tomorrow.strftime('%Y-%m-%d')

    times_lists = [{'from':'00:00','to':'11:59'},{'from':'12:00','to':'23:59'}]

    flights_dict = {
            'ICAO':[],
            'from_city':[],
            'arrival_time':[],
            'airline':[],
            'terminal':[],
            'status_of_flight':[]}
    for i, row in airports_df.iterrows():
        for time in times_lists:

            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{row['ICAO']}/{tomorrow}T{time['from']}/{tomorrow}T{time['to']}"
            querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}
            headers = {
                "X-RapidAPI-Key": "14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4",
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }
            response = requests.get(url, headers=headers, params=querystring)

            my_json = response.json()

            for flight in my_json['arrivals']:
                flights_dict['ICAO'].append(row['ICAO'])
                flights_dict['arrival_time'].append(flight['arrival'].get('scheduledTime',{}).get('local',None))
                flights_dict['from_city'].append(flight['departure'].get('airport',{}).get('name','unknown'))
                flights_dict['airline'].append(flight['airline'].get('name',None))
                flights_dict['terminal'].append(flight['arrival'].get('terminal','unknown'))
                flights_dict['status_of_flight'].append(flight.get('status',None))

    flight_df = pd.DataFrame(flights_dict)
    flight_df['arrival_time'] = flight_df['arrival_time'].str[:-6]
    flight_df.to_sql('flight',con=connection_string,index=False,if_exists='append')

In [ ]:
get_and_store_flights(get_airports())